<a href="https://colab.research.google.com/github/mahluwal/Tensorflow/blob/master/fraud_detection_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')
dataset = pd.read_csv("/content/drive/My Drive/bank_dataset.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dataset

,Unnamed: 0,label,msisdn,aon,daily_decr30,daily_decr90,rental30,rental90,last_rech_date_ma,last_rech_date_da,last_rech_amt_ma,cnt_ma_rech30,fr_ma_rech30,sumamnt_ma_rech30,medianamnt_ma_rech30,medianmarechprebal30,cnt_ma_rech90,fr_ma_rech90,sumamnt_ma_rech90,medianamnt_ma_rech90,medianmarechprebal90,cnt_da_rech30,fr_da_rech30,cnt_da_rech90,fr_da_rech90,cnt_loans30,amnt_loans30,maxamnt_loans30,medianamnt_loans30,cnt_loans90,amnt_loans90,maxamnt_loans90,medianamnt_loans90,payback30,payback90,pcircle,pdate
0,1,0,21408I70789,272,3055.05,3065.15,220.13,260.13,2.0,0.0,1539,2,21.0,3078.0,1539.0,7.50,2,21,3078,1539.0,7.50,0.0,0.0,0,0,2,12,6.0,0.0,2.000000,12,6,0.0,29.000000,29.000000,UPW,7/20/2016
1,2,1,76462I70374,712,12122,12124.75,3691.26,3691.26,20.0,0.0,5787,1,0.0,5787.0,5787.0,61.04,1,0,5787,5787.0,61.04,0.0,0.0,0,0,1,12,12.0,0.0,1.000000,12,12,0.0,0.000000,0.000000,UPW,8/10/2016
2,3,1,17943I70372,535,1398,1398,900.13,900.13,3.0,0.0,1539,1,0.0,1539.0,1539.0,66.32,1,0,1539,1539.0,66.32,0.0,0.0,0,0,1,6,6.0,0.0,1.000000,6,6,0.0,0.000000,0.000000,UPW,8/19/2016
3,4,1,55773I70781,241,21.228,21.228,159.42,159.42,41.0,0.0,947,0,0.0,0.0,0.0,0.00,1,0,947,947.0,2.50,0.0,0.0,0,0,2,12,6.0,0.0,2.000000,12,6,0.0,0.000000,0.000000,UPW,6/6/2016
4,5,1,03813I82730,947,150.6193333,150.6193333,1098.9,1098.9,4.0,0.0,2309,7,2.0,20029.0,2309.0,29.00,8,2,23496,2888.0,35.00,0.0,0.0,0,0,7,42,6.0,0.0,7.000000,42,6,0.0,2.333333,2.333333,UPW,6/22/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189996,189997,0,18013I82733,234,1200,1200,5072.4,5474.34,3.0,0.0,770,1,0.0,770.0,770.0,21.00,1,0,770,770.0,21.00,0.0,0.0,0,0,1,6,6.0,0.0,1.000000,6,6,0.0,0.000000,0.000000,UPW,7/7/2016
189997,189998,1,91352I85340,1260,82293.59,82600.7,6612.94,7250.94,3.0,0.0,8000,6,3.0,63496.0,11874.0,647.40,9,3,99171,11874.0,501.40,0.0,0.0,0,0,3,30,12.0,0.0,4.000000,36,12,0.0,1.500000,1.666667,UPW,7/29/2016
189998,189999,1,87635I84452,1457,5477,9574.42,1730.83,3614.71,7.0,0.0,1539,3,16.0,4617.0,1539.0,26.07,10,8,15390,1539.0,35.57,0.0,0.0,0,0,1,6,6.0,0.0,2738.998602,24,6,0.0,0.000000,5.000000,UPW,8/15/2016
189999,190000,0,58569I90588,555,6229.698667,6256.37,4059.07,11634.55,14.0,0.0,10000,1,0.0,10000.0,10000.0,12.00,1,0,10000,10000.0,12.00,0.0,0.0,0,0,1,6,6.0,0.0,1.000000,6,6,0.0,0.000000,0.000000,UPW,7/22/2016


In [5]:
dataset.columns

Index(['Unnamed: 0', 'label', 'msisdn', 'aon', 'daily_decr30', 'daily_decr90',
       'rental30', 'rental90', 'last_rech_date_ma', 'last_rech_date_da',
       'last_rech_amt_ma', 'cnt_ma_rech30', 'fr_ma_rech30',
       'sumamnt_ma_rech30', 'medianamnt_ma_rech30', 'medianmarechprebal30',
       'cnt_ma_rech90', 'fr_ma_rech90', 'sumamnt_ma_rech90',
       'medianamnt_ma_rech90', 'medianmarechprebal90', 'cnt_da_rech30',
       'fr_da_rech30', 'cnt_da_rech90', 'fr_da_rech90', 'cnt_loans30',
       'amnt_loans30', 'maxamnt_loans30', 'medianamnt_loans30', 'cnt_loans90',
       'amnt_loans90', 'maxamnt_loans90', 'medianamnt_loans90', 'payback30',
       'payback90', 'pcircle', 'pdate'],
      dtype='object')

In [6]:
#some data has "UA" so deleting those rows
dataset = dataset.loc[["UA" not in i for i in dataset["aon"]]]
dataset = dataset.loc[["UA" not in i for i in dataset["daily_decr30"]]]
dataset = dataset.loc[["UA" not in i for i in dataset["daily_decr90"]]]
dataset = dataset.loc[["UA" not in i for i in dataset["rental30"]]]
dataset = dataset.loc[["UA" not in i for i in dataset["rental90"]]]

In [7]:
#Deleting the mobile number data from the dataset
dataset = dataset.drop(columns="msisdn")

In [8]:
## We have left first two columns and taken other columns as input features
X = dataset.iloc[:, 2:].values

# 2nd column is output labels
Y = dataset.iloc[:, 1].values

In [9]:
Y

array([0, 1, 1, ..., 1, 0, 1])

In [10]:
X

array([['272', '3055.05', '3065.15', ..., 29.0, 'UPW', '7/20/2016'],
       ['712', '12122', '12124.75', ..., 0.0, 'UPW', '8/10/2016'],
       ['535', '1398', '1398', ..., 0.0, 'UPW', '8/19/2016'],
       ...,
       ['1457', '5477', '9574.42', ..., 5.0, 'UPW', '8/15/2016'],
       ['555', '6229.698667', '6256.37', ..., 0.0, 'UPW', '7/22/2016'],
       ['617', '13096', '13290.78', ..., 4.0, 'UPW', '7/31/2016']],
      dtype=object)

In [11]:
#Date preprocessing, removing'/' from the date and Reordering it
for i in range(len(X[:, 33])):
  a = X[i, 33].split('/')
  if len(a[0]) == 1:
    a[0] = '0' + a[0]
  if len(a[1]) == 1:
    a[1] = '0' + a[1]
  X[i, 33] = '/'.join(a)
  X[i, 33] = X[i, 33].replace('/', '')[4:]+ X[i, 33].replace('/', '')[:4]

In [12]:
#visualise the date
X[:, 33]

array(['20160720', '20160810', '20160819', ..., '20160815', '20160722',
       '20160731'], dtype=object)

In [13]:
#visualising input data 
X

array([['272', '3055.05', '3065.15', ..., 29.0, 'UPW', '20160720'],
       ['712', '12122', '12124.75', ..., 0.0, 'UPW', '20160810'],
       ['535', '1398', '1398', ..., 0.0, 'UPW', '20160819'],
       ...,
       ['1457', '5477', '9574.42', ..., 5.0, 'UPW', '20160815'],
       ['555', '6229.698667', '6256.37', ..., 0.0, 'UPW', '20160722'],
       ['617', '13096', '13290.78', ..., 4.0, 'UPW', '20160731']],
      dtype=object)

In [14]:
#incoading pcircle in the data because it in str form
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X[:, 32] = labelencoder.fit_transform(X[:, 32])

In [15]:
X[:, 32]

array([0, 0, 0, ..., 0, 0, 0], dtype=object)

In [16]:
X

array([['272', '3055.05', '3065.15', ..., 29.0, 0, '20160720'],
       ['712', '12122', '12124.75', ..., 0.0, 0, '20160810'],
       ['535', '1398', '1398', ..., 0.0, 0, '20160819'],
       ...,
       ['1457', '5477', '9574.42', ..., 5.0, 0, '20160815'],
       ['555', '6229.698667', '6256.37', ..., 0.0, 0, '20160722'],
       ['617', '13096', '13290.78', ..., 4.0, 0, '20160731']],
      dtype=object)

In [17]:
#data type of each column
for i in range(33):
  print(type(X[3, i]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'int'>


In [18]:
#first five columns and 33rd column is string so they cant be normalized
#converting them to float
t = [0,1,2,3,4,33]
for i in t:
  X[:, i] = X[:, i].astype('float64')

In [19]:
#data type of each column
for i in range(33):
  print(type(X[3, i]))

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'int'>
<class 'int'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'int'>


In [20]:
#test train split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 1)

In [21]:
#visualizing test train data
print(len(X_train))
print(len(X_test))
print(len(Y_train))
print(len(Y_test))

138421
46141
138421
46141


In [22]:
#visualizing test train data
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(138421, 34)
(46141, 34)
(138421,)
(46141,)


In [23]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [24]:
X_train

array([[-1.02631723e-01,  7.69964736e-02,  6.22832114e-03, ...,
         4.28506131e-01,  0.00000000e+00,  1.64379250e+00],
       [-9.64429308e-02,  2.87682766e-01,  1.82850636e-01, ...,
         5.49507594e-01,  0.00000000e+00,  3.02953746e-01],
       [ 1.07452920e+01, -5.74582530e-01, -5.48749277e-01, ...,
         9.77687995e-02,  0.00000000e+00, -1.07935425e+00],
       ...,
       [-1.00682188e-01, -5.80163874e-01, -5.53461114e-01, ...,
        -4.18504109e-01,  0.00000000e+00, -1.01023885e+00],
       [-1.06138269e-01, -5.79682799e-01, -5.53054985e-01, ...,
        -4.18504109e-01,  0.00000000e+00, -1.25905429e+00],
       [-1.08414907e-01, -5.51971013e-01, -5.29660368e-01, ...,
        -4.18504109e-01,  0.00000000e+00,  1.92369107e-01]])

In [ ]:
'''from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, Y_train)'''

"from sklearn.ensemble import RandomForestClassifier\nclassifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)\nclassifier.fit(X_train, Y_train)"

In [ ]:
'''from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, Y_train)'''

"from sklearn.tree import DecisionTreeClassifier\nclassifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)\nclassifier.fit(X_train, Y_train)"

In [ ]:
'''from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, Y_train)'''

'from sklearn.naive_bayes import GaussianNB\nclassifier = GaussianNB()\nclassifier.fit(X_train, Y_train)'

In [ ]:
'''from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)'''

"from sklearn.svm import SVC\nclassifier = SVC(kernel = 'rbf', random_state = 0)\nclassifier.fit(X_train, Y_train)"

In [ ]:
"""from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, Y_train)"""

"from sklearn.svm import SVC\nclassifier = SVC(kernel = 'linear', random_state = 0)\nclassifier.fit(X_train, Y_train)"

In [25]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
"""from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, Y_train)"""

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
from sklearn.metrics import confusion_matrix

In [27]:
Y_pred = classifier.predict(X_test)

In [28]:
cm = confusion_matrix(Y_test, Y_pred)

In [29]:
print('confusion matrix =')
print(cm)

confusion matrix =
[[ 2251  3527]
 [ 1769 38594]]


In [30]:
(cm[0][0]+cm[1][1])/(cm[0][0]+ cm[1][0]+cm[0][1]+cm[1][1])

0.8852213866192757

In [31]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(Y_test, Y_pred)
print('accuracy = ')
print(accuracy)

accuracy = 
0.8852213866192757


In [32]:
from sklearn.metrics import recall_score
recall = recall_score(Y_test, Y_pred)
print('recall = ')
print(recall)

recall = 
0.9561727324529892


In [33]:
(1000000-10)/1000000

0.99999

In [34]:
from sklearn.metrics import precision_score
precision = precision_score(Y_test, Y_pred)
print('precision = ')
print(precision)

precision = 
0.9162650459390803


In [35]:
f1_score = 2*recall*precision/(precision + recall)
print('f1_score = ')
print(f1_score)

f1_score = 
0.9357936084573978
